In [44]:
import random
import sys
import tkinter as tkinter
from tkinter import filedialog as tkfiledialog

ENTRY_WIDTH = 20  # width of text entry field

In [45]:
import sys, importlib
print("sys.executable:", sys.executable)
print("sys.version:", sys.version)
print("pandas available:", importlib.util.find_spec("pandas") is not None)
# safe install using the kernel's interpreter
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])
%pip install pandas

sys.executable: /usr/local/bin/python3
sys.version: 3.14.2 (v3.14.2:df793163d58, Dec  5 2025, 12:18:06) [Clang 16.0.0 (clang-1600.0.26.6)]
pandas available: True


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
No module named pip
45017.46s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [46]:
import sys, importlib
print("sys.executable:", sys.executable)
print("sys.version:", sys.version)
print("pandas available:", importlib.util.find_spec("pandas") is not None)

sys.executable: /usr/local/bin/python3
sys.version: 3.14.2 (v3.14.2:df793163d58, Dec  5 2025, 12:18:06) [Clang 16.0.0 (clang-1600.0.26.6)]
pandas available: True


In [47]:
SIM_THRESHOLD = 0.8
import pandas as pd
sim = pd.read_csv("/Users/abbyhultquist/Desktop/First Year Project/nouns.csv", index_col=0)


# builds a list of edges where similarity >= 0.9, in the variable `edges`
edges = []
for w1 in sim.index:
    for w2 in sim.columns:
        if w1 != w2 and sim.loc[w1, w2] >= SIM_THRESHOLD:
            edges.append((w1, w2))

In [48]:
# load (no index_col) or reset index if already loaded with index_col=0
cdi = pd.read_csv("/Users/abbyhultquist/Desktop/First Year Project/CDI_cleaned.csv")
# or if already read with index_col=0: cdi = cdi.reset_index()

# normalize and coerce key columns
cdi.columns = cdi.columns.str.strip()
cdi['child_id'] = pd.to_numeric(cdi['child_id'], errors='coerce')
cdi['session_num'] = pd.to_numeric(cdi['session_num'], errors='coerce')

child_id = 4139
session_num = 1

mask = (cdi['child_id'] == int(child_id)) & (cdi['session_num'] == int(session_num))
print("matches:", mask.sum(), "matching indices:", cdi.index[mask].tolist())

rows = cdi.loc[mask]
if rows.empty:
    raise ValueError(f"No rows match child_id={child_id} session_num={session_num}")
row = rows.iloc[0]
print(row.to_dict())

row = cdi[
    (cdi.child_id == child_id) &
    (cdi.session_num == session_num)
].iloc[0]

# CDI word columns start after metadata
metadata_cols = [
    "child_id","study id","study","gender","age","birthday",
    "session_date","session_num","total num sessions",
    "words_spoken","items excluded","percentile",
    "extra categories","revision","languages",
    "num languages","cdi type","hard of hearing","deleted", "child_column_id", 
    "first_session_num", "late_talker","last_session_num",	"late_bloomer",	"plt",	"class",	
    "typical_talker",	"group"
]

word_cols = [c for c in cdi.columns if c not in metadata_cols]

known_words = {w for w in word_cols if row[w] == 1}

matches: 1 matching indices: [0]
{'child_id': 4139.0, 'study id': 'LTP102', 'study': 'Long-1', 'gender': 'F', 'age': 16.99539776462853, 'birthday': '2008/10/17', 'session_date': '2010/03/18', 'session_num': 1, 'total num sessions': 12, 'words_spoken': 94, 'items excluded': 0, 'percentile': 63.6363636363636, 'extra categories': 2, 'revision': 0, 'languages': 'english', 'num languages': 1, 'cdi type': 'fullenglishmcdi', 'hard of hearing': 0, 'deleted': 0, 'baa baa': 1, 'choo choo': 0, 'cockadoodledoo': 0, 'grr': 0, 'meow': 0, 'moo': 1, 'ouch': 0, 'quack quack': 0, 'uh oh': 0, 'vroom': 0, 'woof woof': 0, 'yum yum': 1, 'alligator': 0, 'animal': 0, 'ant': 0, 'bear': 1, 'bee': 0, 'bird': 1, 'bug': 0, 'bunny': 0, 'butterfly': 0, 'cat': 1, 'chicken(animal)': 0, 'cow': 0, 'deer': 0, 'dog': 1, 'donkey': 0, 'duck': 1, 'elephant': 0, 'fish(animal)': 1, 'frog': 1, 'giraffe': 0, 'goose': 0, 'hen': 0, 'horse': 0, 'kitty': 0, 'lamb': 0, 'lion': 0, 'monkey': 0, 'moose': 0, 'mouse': 0, 'owl': 0, 'pengui

In [49]:
# count known words
num = len(known_words)
print("known_words count:", num)

num_tot = len(word_cols)
print("total words count:", num_tot)


known_words count: 94
total words count: 680


In [50]:
import itertools

# build list of edges where similarity >= SIM_THRESHOLD, in the variable `child_edges`
child_edges = []
for w1, w2 in itertools.combinations(sorted(known_words), 2):
    try:
        if sim.loc[w1, w2] >= SIM_THRESHOLD:    # similarity threshold
            child_edges.append((w1, w2)) # add to list if similarity >= threshold
    except KeyError:
        # skip if either word not found in similarity matrix
        continue

print("child_edges count:", len(child_edges))
print("sample edges:", child_edges[:10])


child_edges count: 3
sample edges: [('banana', 'bread'), ('bread', 'cookie'), ('hat', 'shirt')]


In [51]:
all_words_df = pd.DataFrame(edges, columns=["word1", "word2"])
print("all_words edges:", len(all_words_df))
print(all_words_df.head())

# write to file (NO header, NO index)
all_words_path = "/Users/abbyhultquist/Desktop/First Year Project/all_words.txt"
all_words_df.to_csv(all_words_path, index=False, header=False)

print("Saved all_words.txt to:", all_words_path)


all_words edges: 188
   word1       word2
0  ankle      street
1   aunt  babysitter
2   aunt        lady
3   aunt         man
4   aunt       woman
Saved all_words.txt to: /Users/abbyhultquist/Desktop/First Year Project/all_words.txt


In [52]:
child_vocab_df = pd.DataFrame(child_edges, columns=["word1", "word2"])

print("child_vocab edges:", len(child_vocab_df))
print(child_vocab_df.head())

child_vocab_path = f"/Users/abbyhultquist/Desktop/First Year Project/child_vocab_{child_id}_s{session_num}.txt"

child_vocab_df.to_csv(child_vocab_path, index=False, header=False)

print("Saved child vocabulary file to:", child_vocab_path)


child_vocab edges: 3
    word1   word2
0  banana   bread
1   bread  cookie
2     hat   shirt
Saved child vocabulary file to: /Users/abbyhultquist/Desktop/First Year Project/child_vocab_4139_s1.txt
